# Question 3

## Question 3.1.1

The program is in the next block.

## Question 3.1.2

## Question 3.2.1

The program is in the third block.

## Question 3.2.2

## Question 3.2.3

In [ ]:
# Q 3.1.1

In [ ]:
# Q 3.2.1

# Question 4

## Question 4.1.1

## Question 4.1.2

## Question 4.1.3

## Question 4.2.4

The program is in the next block.

## Question 4.2.5


In [ ]:
# Q 4.2.4

# Question 5

## Question 5.1

The program is in the next block.

## Question 5.2.1

## Question 5.2.2


In [ ]:
# Q 5